<a href="https://colab.research.google.com/github/Stonetenth2005/Machine-Learning/blob/main/11-Disctretization/binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer

from sklearn.metrics import accuracy_score

In [29]:
df = pd.read_csv('/content/Titanic-Dataset.csv')[['Age', 'Fare', 'SibSp', 'Parch', 'Survived']]
df.head()

,Age,Fare,SibSp,Parch,Survived
0,22.0,7.2500,1,0,0
1,38.0,71.2833,1,0,1
2,26.0,7.9250,0,0,1
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0


In [30]:
df.dropna(inplace=True)

In [31]:
df['Family'] = df['SibSp'] + df['Parch']
df.drop(columns=['SibSp', 'Parch'], inplace=True)
df.head()

,Age,Fare,Survived,Family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


# Without Binarization

In [32]:
X = df.iloc[:,[0,1,3]]
Y = df['Survived']

In [33]:
X_test, X_train, Y_test, Y_train = train_test_split(X, Y, test_size=0.2)

In [34]:
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)
accuracy_score(Y_test, Y_pred)

0.5989492119089317

In [35]:
np.mean(cross_val_score(DecisionTreeClassifier(), X, Y, scoring='accuracy'))

np.float64(0.65124593716143)

# With Binarization

In [36]:
from sklearn.preprocessing import Binarizer

In [38]:
trf = ColumnTransformer(transformers = [
    ('bin', Binarizer(copy=False), ['Family'])
], remainder='passthrough')

X_train_transformed = trf.fit_transform(X_train)
X_test_transformed = trf.transform(X_test)

In [39]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed, Y_train)

Y_pred = clf.predict(X_test_transformed)
accuracy_score(Y_test, Y_pred)

0.5726795096322241

In [41]:
X_tranformed = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(), X_tranformed, Y, scoring='accuracy'))

np.float64(0.6232443612725304)

In [45]:
pd.DataFrame({
    'Age': X_tranformed[:, 1],
    'Fare': X_tranformed[:, 2],
    'Family': X_tranformed[:, 0],
})

,Age,Fare,Family
0,22.0,7.2500,1.0
1,38.0,71.2833,1.0
2,26.0,7.9250,0.0
3,35.0,53.1000,1.0
4,35.0,8.0500,0.0
...,...,...,...
709,39.0,29.1250,1.0
710,27.0,13.0000,0.0
711,19.0,30.0000,0.0
712,26.0,30.0000,0.0
